#Hackathon - Second Quantum Computing School
##Group: QNat
Alberto Bezerra de Palhares Junior   
Joab Morais Varela  
Moisés da Rocha Alves  
Paulo Vitor de Queiroz Ferreira  
Tailan Santos Sarubi
#Introduction to Applications of Quantum Computing to Quantum Chemistry
##Challenge: $b$



Firstly we need to import the libraries we are going to use.

In [1]:
!pip install pennylane -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00


In [2]:
#Author: Alberto Palhares

import pennylane as qml
from pennylane import numpy as pnp

Our system is a spin chain of $N=4$ spin $s=1/2$ particles where each spin can point in the positive or negative z-direction, and there is also an external transverse magnetic field being applied.

The Hamiltonian we are going to minimize is:

$$H = -\sum_{i=1}^N Z_i \otimes Z_{i+1} - h\sum_{i=1}^{N}X_i$$,

where $X_i$ and $Z_i$ are respectively Pauli gate X and Z applied to the i-th qubit in the quantum circuit, and $h$ is the intensity of the transversal magnetic field.

In [3]:
num_qubits=4
# Function which returns our Hamiltonian
def hamiltonian(hfield, N=num_qubits):
  coeffs = []
  obs=[]
  # First term of the Hamiltonian
  for i in range(N):
    coeffs.append(-1)
    if (i == N - 1):
      obs.append(qml.PauliZ(i) @ qml.PauliZ(0))
    else:
      obs.append(qml.PauliZ(i) @ qml.PauliZ(i+1))
  # Second term of the Hamiltonian
  for j in range(N):
    coeffs.append(-hfield)
    obs.append(qml.PauliX(j))

  return qml.Hamiltonian(coeffs, obs)

Testing our Hamiltonian function:

In [4]:
#Inform the number of qubits here ##############################################
hfield = 5
################################################################################
h_c = hamiltonian(hfield)
print("H = ", h_c)

H =  -1 * (Z(0) @ Z(1)) + -1 * (Z(1) @ Z(2)) + -1 * (Z(2) @ Z(3)) + -1 * (Z(3) @ Z(0)) + -5 * X(0) + -5 * X(1) + -5 * X(2) + -5 * X(3)


Before using a VQA to solve our problem, it is good to check the correct result, we could diagonilize our Hamiltonian to find its ground state, which will serve as theoretical value to compare with our vqa result.

In [5]:
# Here is the Hamiltonian Matrix to be diagonalized.
obs = qml.matrix(hamiltonian(hfield))
print("H = ", obs)

H =  [[-4.+0.j -5.+0.j -5.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j -5.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-5.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j
  -5.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-5.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j
   0.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -5.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j
   0.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-5.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -5.+0.j -5.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -5.+0.j  0.+0.j  0.+0.j -5.+0.j  4.+0.j  0.+0.j -5.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -5.+0.j  0.+0.j -5.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -5.+0.j  0.+0.j -5.+0.j -5.+0.j  0.+0.

In [ ]:
# Using numpy.linalg to diagonalize our matrix
eigv = pnp.linalg.eigvals(obs)

# Getting the minimum energy from the eingenvalues
ground_state_energy_diag = pnp.min(pnp.real(eigv)) #tirar esse round dps
print("Ground state energy computed with diagonalization: ", ground_state_energy_diag)

Ground state energy computed with diagonalization:  -20.202968496019384


Now we want to use a Variational algorithm to find the ground state of this Hamiltonian. We will use VQE to solve  this task. In order to do this, we need to define an Ansatz to minimize our Hamiltonian. The chosen ansatz was the Efficient SU(2): https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.EfficientSU2. We chose it because it is a well-known ansatz that results a fairly good answer.

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)

# Define the efficient ansatz
def efficient_ansatz(params):
    for i in range(num_layers):
        for w in range(num_qubits):
            qml.RY(params[2 * w], wires=w)
            qml.RZ(params[2 * w + 1], wires=w)
        for w in range(num_qubits - 1):
            qml.CNOT(wires=[w, w + 1])
        # Add an entangling layer
        qml.CNOT(wires=[num_qubits - 1, 0])

# Create a quantum circuit that returns the exp value of our Hamiltonian
@qml.qnode(dev)
def circuit(params):
    efficient_ansatz(params)
    return qml.expval(h_c)

num_layers = 10  # VQE's adjustable parameter that deepens our quantum circuit to better approximate the answer.
param_size = num_layers * num_qubits * 2

params = pnp.zeros(param_size)
angles = [params] # Store the values of the circuit parameter
cost = [circuit(params)] # Store the values of the cost function


opt = qml.AdamOptimizer(stepsize=0.01, beta1=0.9, beta2=0.99, eps=1e-08) # Our optimizer! we chose Adam because it is a well-known algorithm.
max_iterations = 200 # Maximum number of calls to the optimizer
conv_tol = 1e-08 # Convergence threshold to stop our optimization procedure

for n in range(max_iterations):
    params, prev_cost = opt.step_and_cost(circuit, params)
    cost.append(circuit(params))
    angles.append(params)

    conv = pnp.abs(cost[-1] - prev_cost)
    if n % 10 == 0: # print the optimization process
        print("Step = ", n,",  Cost function = ", cost[-1])
    if conv <= conv_tol:
        break

ground_state_energy_vqe = cost[-1]
print("Final value of the cost function =", ground_state_energy_vqe)

Step =  0 ,  Cost function =  -4.438880597631957
Step =  10 ,  Cost function =  -11.955274870784288
Step =  20 ,  Cost function =  -18.862563862788495
Step =  30 ,  Cost function =  -19.47471083242909
Step =  40 ,  Cost function =  -19.452504492058623
Step =  50 ,  Cost function =  -19.820663108291296
Step =  60 ,  Cost function =  -19.775118307457518
Step =  70 ,  Cost function =  -19.821951121368706
Step =  80 ,  Cost function =  -19.818323323453782
Step =  90 ,  Cost function =  -19.824697745302895
Step =  100 ,  Cost function =  -19.82387851754341
Step =  110 ,  Cost function =  -19.82475825443246
Step =  120 ,  Cost function =  -19.824704568049782
Step =  130 ,  Cost function =  -19.82475806439189
Step =  140 ,  Cost function =  -19.824787650939317
Step =  150 ,  Cost function =  -19.82478364892697
Final value of the cost function = -19.824788134483192


In [ ]:
print("The relative error between the two Hamiltonian values: ", (ground_state_energy_diag - ground_state_energy_vqe)/ground_state_energy_diag)

The relative error between the two Hamiltonian values:  0.018719049213520526
